In [3]:
import sys
sys.version

'3.10.8 (main, Nov 24 2022, 08:09:04) [Clang 14.0.6 ]'

In [4]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.xkcd()

import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import plotnine as p9
import statsmodels.api as sm
import math

pd.options.mode.chained_assignment = None 

In [5]:
from functions import *

In [4]:
def filter_by_sector(tickers_list_,new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date):

    for ticker in tickers_list_:

        if ticker in tickers_remove_list:
            continue

        df_fc = quandl.get_table('ZACKS/FC',ticker = ticker, per_end_date = {'gte':start_date,'lte':end_date},qopts={"columns":"zacks_sector_code"})

        if len(df_fc)!=0:
            sector_code_list = list(df_fc['zacks_sector_code'])

            if bool(set(zacks_sector_code_list) & set(sector_code_list))==False:
                new_ticker_list.append(ticker)

    return new_ticker_list

In [5]:
start_date = '2014-06-30'
end_date = '2022-06-30'

df_tickers = pd.read_csv("zacks-tickers.csv")
tickers_list = list(df_tickers['3262Q'])

zacks_sector_code_list = [5,13]
tickers_remove_list = ['ABEO']

new_ticker_list = []
new_ticker_list = filter_by_sector(tickers_list[3000:4000],new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date)

# new_ticker_list

In [6]:
df_new_ticker4 = pd.DataFrame(new_ticker_list, columns =['ticker'])

In [7]:
df_new_ticker4
    

,ticker
0,FDC
1,FDO
2,FDP
3,FDS
4,FDX
...,...
504,IACAF
505,IAG
506,IAGX
507,IART


In [8]:
df_new_ticker4.to_csv('filter_by_sector_4000.csv',index=False)

In [9]:
new_ticker_list8 = []
new_ticker_list8 = filter_by_sector(tickers_list[7000:],new_ticker_list,zacks_sector_code_list,tickers_remove_list,start_date,end_date)

df_new_ticker8 = pd.DataFrame(new_ticker_list, columns =['ticker'])
df_new_ticker8.to_csv('filter_by_sector_8000.csv',index=False)

In [10]:
len(df_new_ticker8)

1610

In [14]:
def filter_by_adj_close(tickers_list_,start_date,end_date,num):

    filter_by_adj_close_list = []

    for ticker in tickers_list_:
        df1 = get_adj_close(ticker,start_date,end_date)
        if len(df1) == num:
            filter_by_adj_close_list.append(ticker)

    return filter_by_adj_close_list


start_date = '2014-06-30'
end_date = '2022-06-30'

df8 = pd.read_csv("filter_by_sector_8000.csv")
ticker_list8 = list(df8['ticker'])
print(len(ticker_list8))

filter_by_adj_close_list8 = filter_by_adj_close(ticker_list8,start_date,end_date,2016)

1610


In [15]:
df_new_ticker8 = pd.DataFrame(filter_by_adj_close_list8, columns =['ticker'])
df_new_ticker8.to_csv('filter_by_adj_close_8000.csv',index=False)

In [16]:
len(df_new_ticker8)

624

In [6]:
def filter_by_debt_ratio(df_ticker,start_date,end_date):

    df_ticker['debt_ratio_greater_than_0.1'] = 0

    for i in range(len(df_ticker)):

        ticker = df_ticker['ticker'][i]

        df_zack = generate_zacks_data(ticker,start_date,end_date)
        df_price = get_adj_close(ticker,start_date,end_date)

        df = df_price.join(df_zack,how='outer')
        df['adj_close'] =  df['adj_close'].fillna(method='ffill')

        df1 = debt_to_market_cap(df,df_zack)

        count = df1[['debt_to_market_cap']][df1[['debt_to_market_cap']]  > 0.1].count()

        df_ticker['debt_ratio_greater_than_0.1'][i] = count

        # if i==100:
        #     break

    return df_ticker

In [9]:
start_date = '2014-06-30'
end_date = '2022-06-30'

df_ticker_adj_close = pd.read_csv("filter_by_adj_close.csv")
print(len(df_ticker_adj_close))

df_ticker_adj_close = df_ticker_adj_close[1500:]
df_ticker_adj_close.reset_index(inplace=True)
print(len(df_ticker_adj_close))

df_ticker_debt_ratio4 = filter_by_debt_ratio(df_ticker_adj_close,start_date,end_date)

1975
475


In [10]:
df_ticker_debt_ratio4.to_csv('filter_by_debt_ratio_4.csv',index=False)